# Example Experiment
> Experiment using Repeated MNIST and BatchBALD vs BALD vs random sampling

This notebook ties everything together and runs an AL loop.

In [1]:
# experiment

#import blackhc.project.script
from tqdm.auto import tqdm

In [2]:
# experiment

import math
import time
import torch
from torch import nn as nn
from torch.nn import functional as F

from batchbald_redux import (
    active_learning,
    batchbald,
    consistent_mc_dropout,
    joint_entropy,
    repeated_mnist,
)
torch.manual_seed(0)


Let's define our Bayesian CNN model that we will use to train MNIST.

In [3]:
# experiment


class BayesianCNN(consistent_mc_dropout.BayesianModule):
    def __init__(self, num_classes=10):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv1_drop = consistent_mc_dropout.ConsistentMCDropout2d()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv2_drop = consistent_mc_dropout.ConsistentMCDropout2d()
        self.fc1 = nn.Linear(1024, 128)
        self.fc1_drop = consistent_mc_dropout.ConsistentMCDropout()
        self.fc2 = nn.Linear(128, num_classes)

    def mc_forward_impl(self, input: torch.Tensor):
        input = F.relu(F.max_pool2d(self.conv1_drop(self.conv1(input)), 2))
        input = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(input)), 2))
        input = input.view(-1, 1024)
        input = F.relu(self.fc1_drop(self.fc1(input)))
        input = self.fc2(input)
        input = F.log_softmax(input, dim=1)

        return input

In [4]:
from torchvision.datasets import MNIST,FashionMNIST,CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import wandb

In [5]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = FashionMNIST("data", train=True, download=True,transform=transform)
test_dataset = FashionMNIST("data", train=False, download=True,transform=transform)


In [6]:
# experiment

#train_dataset, test_dataset = repeated_mnist.create_repeated_MNIST_dataset(num_repetitions=1, add_noise=False)
train_dataset,monitor_set = random_split(
    train_dataset, [59900,100]
)
num_initial_samples = 20
num_classes = 10

initial_samples = active_learning.get_balanced_sample_indices(
    repeated_mnist.get_targets(train_dataset), num_classes=num_classes, n_per_digit=num_initial_samples / num_classes
)

In [7]:
# experiment
max_training_samples = 500
acquisition_batch_size = 5
num_inference_samples = 100
num_test_inference_samples = 5
num_samples = 100000

test_batch_size = 512
batch_size = 64
scoring_batch_size = 128
training_iterations = 4096 * 6

use_cuda = torch.cuda.is_available()

print(f"use_cuda: {use_cuda}")
device = "cuda" if use_cuda else "cpu"
#device="cpu"
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, **kwargs)

active_learning_data = active_learning.ActiveLearningData(train_dataset)

# Split off the initial samples first.
active_learning_data.acquire(initial_samples)

# THIS REMOVES MOST OF THE POOL DATA. UNCOMMENT THIS TO TAKE ALL UNLABELLED DATA INTO ACCOUNT!
active_learning_data.extract_dataset_from_pool(50000)

train_loader = torch.utils.data.DataLoader(
    active_learning_data.training_dataset,
    sampler=active_learning.RandomFixedLengthSampler(active_learning_data.training_dataset, training_iterations),
    batch_size=batch_size,
    **kwargs,
)

pool_loader = torch.utils.data.DataLoader(
    active_learning_data.pool_dataset, batch_size=scoring_batch_size, shuffle=False, **kwargs
)


monitor_loader = torch.utils.data.DataLoader(
    monitor_set, batch_size=scoring_batch_size, shuffle=False, **kwargs
)



use_cuda: True


In [9]:
from csv import writer


In [10]:
wandb.init(project="ActiveLearning_1124_FASHION")
wandb.run.name = 'BALD_CIFAR_FULL1125_10000_monitor'
wandb.run.save()

added_indices = []

pbar = tqdm(initial=len(active_learning_data.training_dataset), total=max_training_samples, desc="Training Set Size")
epoch=0
model = BayesianCNN(num_classes).to(device=device)
optimizer = torch.optim.Adam(model.parameters())

with open('bald_data_history.csv', 'a', newline='') as csv_obj:

        writer_object = writer(csv_obj)
        writer_object.writerow([x[1] for  x in monitor_set])
        csv_obj.close()
while True:

    epoch+=1
    model.train()
    train_loss=0
    # Train
    for data, target in tqdm(train_loader, desc="Training", leave=False):
        data = data.to(device=device)
        target = target.to(device=device)

        optimizer.zero_grad()

        prediction = model(data, 1).squeeze(1)
        loss = F.nll_loss(prediction, target)
        train_loss+=loss.item()
        loss.backward()
        optimizer.step()
                                                   
    train_loss /= len(train_loader)
    # Test
    loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader, desc="Testing", leave=False):
            data = data.to(device=device)
            target = target.to(device=device)

            prediction = torch.logsumexp(model(data, num_test_inference_samples), dim=1) - math.log(
                num_test_inference_samples
            )
            loss += F.nll_loss(prediction, target, reduction="sum")
        
            prediction = prediction.max(1)[1]
            correct += prediction.eq(target.view_as(prediction)).sum().item()

    loss /= len(test_loader.dataset)

    percentage_correct = 100.0 * correct / len(test_loader.dataset)


                                                   
    if len(active_learning_data.training_dataset) >= max_training_samples:
        break

    # Acquire pool predictions
                                                   
    acquir_start=time.time()
    N = len(active_learning_data.pool_dataset)
    logits_N_K_C = torch.empty((N, num_inference_samples, num_classes), dtype=torch.double, pin_memory=use_cuda)
    
    with torch.no_grad():
        model.eval()
        
        for i, (data, _) in enumerate(tqdm(pool_loader, desc="Evaluating Acquisition Set", leave=False)):
            data = data.to(device=device)

            lower = i * pool_loader.batch_size
            upper = min(lower + pool_loader.batch_size, N)
            logits_N_K_C[lower:upper].copy_(model(data, num_inference_samples).double(), non_blocking=True)

            
    with torch.no_grad():
#         candidate_batch = batchbald.get_batchbald_batch(
#             logits_N_K_C, acquisition_batch_size, num_samples, dtype=torch.double, device=device
#         )
         candidate_batch = batchbald.get_bald_batch(
             logits_N_K_C, acquisition_batch_size, dtype=torch.double, device=device
         )
                                                   
    acquire_end=acquir_start-time.time()
    wandb.log({"Epoch":epoch,
               "Num_Pool": len(active_learning_data.pool_dataset),
               "Num_Train":len(active_learning_data.training_dataset),
               "Test Loss": loss,
               'Train Loss': train_loss,
              "Test Accuracy":percentage_correct,
               "Query Time":-acquire_end})

                                                   
    targets = repeated_mnist.get_targets(active_learning_data.pool_dataset)
    dataset_indices = active_learning_data.get_dataset_indices(candidate_batch.indices)

    print("Dataset indices: ", dataset_indices)
    print("Scores: ", candidate_batch.scores)
    print("Labels: ", targets[candidate_batch.indices])

                                                   
    
    
    
    active_learning_data.acquire(candidate_batch.indices)
    added_indices.append(dataset_indices)
    pbar.update(len(dataset_indices))
    
    #Monitoring Function
    N = len(monitor_set)
    logits_N_K_C = torch.empty((N, num_inference_samples, num_classes), dtype=torch.double, pin_memory=use_cuda)
    with torch.no_grad():
        model.eval()

        for i, (data, _) in enumerate(monitor_loader):
            data = data.to(device=device)

            lower = i * pool_loader.batch_size
            upper = min(lower + pool_loader.batch_size, N)
            logits_N_K_C[lower:upper].copy_(model(data, num_inference_samples).double(), non_blocking=True)


        with torch.no_grad():
             monitor_batch = batchbald.get_bald_batch(
                 logits_N_K_C, N, dtype=torch.double, device=device
             )


    with open('bald_data_history.csv', 'a', newline='') as csv_obj:

        writer_object = writer(csv_obj)
        writer_object.writerow(monitor_batch.scores)

        csv_obj.close()

wandb: Currently logged in as: hslrock (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Dataset indices:  [13117 16516 10145 39578 30893]
Scores:  [1.3577637849649844, 1.2985060082557935, 1.268627365360672, 1.2558393956713187, 1.2239118007430252]
Labels:  tensor([2, 5, 8, 5, 8])


Dataset indices:  [12764 49657 21641 45809 36317]
Scores:  [1.1686313222574152, 1.149420109077686, 1.137297874818871, 1.112525762453079, 1.099909493690224]
Labels:  tensor([8, 2, 6, 6, 8])


Dataset indices:  [52735 45694 22423 38670 44142]
Scores:  [1.3786215089262497, 1.2548421343621503, 1.2302711452038455, 1.2230333282284926, 1.2205505060353796]
Labels:  tensor([3, 5, 5, 5, 5])


...

Dataset indices:  [12965 23722 38908 39817 45916]
Scores:  [1.0555347936060309, 1.0121650433287448, 1.001869495038454, 0.9991892137577744, 0.9701950629181719]
Labels:  tensor([2, 0, 2, 8, 8])


Dataset indices:  [ 5391 13363 18440  9295 31787]
Scores:  [1.077252178152583, 0.9986337958086982, 0.9871599831527358, 0.983564357348754, 0.9767734511393166]
Labels:  tensor([4, 4, 6, 3, 3])


Dataset indices:  [27542 15096 15581 48354 54053]
Scores:  [1.0514720978662881, 1.0108089345440496, 1.0083004635599804, 1.0066091486786626, 1.0059856308016772]
Labels:  tensor([0, 0, 6, 8, 1])


Dataset indices:  [ 8954  2817 59490 47923  8975]
Scores:  [1.0773357447279883, 1.0553657723365175, 1.0284190737668846, 1.0272402408846049, 1.0181138385389756]
Labels:  tensor([8, 2, 5, 5, 4])


Dataset indices:  [57010 21334 19446 44576  2059]
Scores:  [1.0139235222419676, 1.0106500905591118, 1.0105567037139151, 0.9890762050324047, 0.982838780011821]
Labels:  tensor([7, 7, 3, 5, 3])
